# Automatic Centerlines Extraction


First, extract the coronary artery centerlines from your dataset using the method described in 
[Coronary artery centerline extraction in cardiac CT angiography using a CNN-based orientation classifier (Wolternik et al., 2019b)](https://www.sciencedirect.com/science/article/pii/S1361841518308491). For now, CCTA scans from `<dataset_dir>` are matched with the patterns `*.mhd`
and `*_image.nii.gz`. 

```bash
source scripts/centerlines_autoextract.sh <dataset_dir> <output_dir>
```

For every `datapoint_X` in `<dataset_dir>`, this command creates files `<output_dir>/<datapoint_X>/vessel0.txt`,
`<output_dir>/<datapoint_X>/vessel1.txt` etc., where each `vessel<N>.txt` file corresponds to a discovered coranary 
artery vessel and it contains an array of 4D points (3D XYZ coordinates + artery lumen radius estimate).

## Create the HD5-encoded Dataset

The HD5 file type allows on-the-fly loading of big datasets, without the need to load entire arrays in memory -- 
relevant parts are loaded as needed, reducing memory footprint. 

For more inforamation, refer to the 
[HD5 Python documentation](https://docs.h5py.org/en/stable/).

In [ ]:
%cd ../

/home/marco/contrast-gan-3D


Adjust the variables in the following cell to match your setup:

In [ ]:
from pathlib import Path

DATASET_DIR = Path("/home/marco/data/MMWHS/ct_test")

OUTPUT_DIR = DATASET_DIR / "auto_centerlines"

REORIENT_RPS = "MMWHS" in str(DATASET_DIR)

# By default, .h5 converted files are saved inside `datset_dir`; set this
# variable if you want to save them somewhere else
H5_OUTPUT_DIR = None

print(str(DATASET_DIR))
print(str(OUTPUT_DIR))
print(str(H5_OUTPUT_DIR))
print(REORIENT_RPS)

/home/marco/data/MMWHS/ct_test
/home/marco/data/MMWHS/ct_test/auto_centerlines
None
True


In [ ]:
from pprint import pprint

cctas = sorted(
    list(DATASET_DIR.glob("*.mhd")) + list(DATASET_DIR.glob("*_image.nii.gz"))
)
centerlines = sorted([d for d in OUTPUT_DIR.glob("*") if d.is_dir()])
print(f"Found {len(cctas)} CCTAs with {len(centerlines)} centerlines")

zipped = list(zip(cctas, centerlines))
pprint(zipped[:3])

Found 40 CCTAs with 40 centerlines
[(PosixPath('/home/marco/data/MMWHS/ct_test/ct_test_2001_image.nii.gz'),
  PosixPath('/home/marco/data/MMWHS/ct_test/auto_centerlines/ct_test_2001_image.nii.gz')),
 (PosixPath('/home/marco/data/MMWHS/ct_test/ct_test_2002_image.nii.gz'),
  PosixPath('/home/marco/data/MMWHS/ct_test/auto_centerlines/ct_test_2002_image.nii.gz')),
 (PosixPath('/home/marco/data/MMWHS/ct_test/ct_test_2003_image.nii.gz'),
  PosixPath('/home/marco/data/MMWHS/ct_test/auto_centerlines/ct_test_2003_image.nii.gz'))]


In [ ]:
from contrast_gan_3D.utils import io_utils, logging_utils

logging_utils.set_project_loggers_level(level="DEBUG")

contrast_gan_3D.utils.io_utils: INFO -> DEBUG


In [ ]:
for ccta, centerlines_dir in zipped:
    if io_utils.stem(centerlines_dir) == io_utils.stem(ccta):
        io_utils.sitk_to_h5(
            ccta,
            centerlines_dir,
            centerlines_dir / "ostia.xml",
            h5_output_dir=H5_OUTPUT_DIR,
            reorient_RPS=REORIENT_RPS,
        )
    else:
        print(f"**No centerlines for {str(ccta)!r}**")

[2024-01-18 15:44:45,200: DEBUG] dtype conversion float32->int16 (contrast_gan_3D.utils.io_utils:29)
[2024-01-18 15:44:45,226: DEBUG] Clipped to (-1024, 3072) (contrast_gan_3D.utils.io_utils:33)


`data_array` is not of type `MetaTensor, assuming affine to be identity.


[2024-01-18 15:44:46,830: INFO] REORIENT RPS: '/home/marco/data/MMWHS/ct_test/ct_test_2001_image.nii.gz' (contrast_gan_3D.utils.io_utils:140)
[2024-01-18 15:44:46,884: DEBUG] Saved centerlines to '/home/marco/data/MMWHS/ct_test/auto_centerlines/ct_test_2001_image.nii.gz/centerlines.npz' with key 'centerlines' (contrast_gan_3D.utils.io_utils:89)
[2024-01-18 15:44:46,908: DEBUG] CCTA: torch.Size([512, 512, 224]) centerlines: (6030, 4) ostia: (2, 3) (contrast_gan_3D.utils.io_utils:147)
[2024-01-18 15:44:46,909: DEBUG] H5 file: '/home/roel/data/MMWHS/ct_test/ct_test_2001_image.h5' (contrast_gan_3D.utils.io_utils:156)
[2024-01-18 15:44:50,735: DEBUG] dtype conversion float32->int16 (contrast_gan_3D.utils.io_utils:29)
[2024-01-18 15:44:50,754: DEBUG] Clipped to (-1024, 3072) (contrast_gan_3D.utils.io_utils:33)
[2024-01-18 15:44:52,142: INFO] REORIENT RPS: '/home/marco/data/MMWHS/ct_test/ct_test_2002_image.nii.gz' (contrast_gan_3D.utils.io_utils:140)
[2024-01-18 15:44:52,244: DEBUG] Saved cen